# Biophysical self-organization amplifies the resilience of salt marshes

The tensile stress, humidity of soil surface, water content, and plant biomass are defined as 𝑆(𝒓, 𝑡), 𝐸(𝒓, 𝑡), 𝑊(𝒓, 𝑡), and 𝑃(𝒓, 𝑡) at position 𝒓 = (𝑥, 𝑦) and time 𝑡, respectively. 

$\frac{\partial S}{\partial t} = r_S (S-S_{min}) (S_{critical}-S) (S-S_{max}) + c_S (E - e)$

$\frac{\partial W}{\partial t} = \frac{c_W}{k_W + S} - d_E (E - e) W - l_P P \frac{W}{k_P+W} + D_W\mathrm{\Delta W}$

$\frac{\partial E}{\partial t} = r_E (W - h) (E - e) + D_E \mathrm{\Delta E}$

$\frac{\partial P}{\partial t} = r_P \frac{W(1-S)}{k_P+W} P - d_P P^2 + D_P \mathrm{\Delta P}$

In [ ]:
%reset -f

### Loading some crucial python packages

In [ ]:
import time,os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# Widening the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
FS=25
rcParams['figure.dpi'] = 5000
rcParams['savefig.dpi'] = 5000
rcParams['font.size'] = FS
rcParams['font.family'] = 'StixGeneral'#'Times New Roman'
rcParams["mathtext.fontset"] = 'stix'
# rcParams["mathtext.default"] = "rm" 
#rcParams['xtick.direction'] = 'in'
plt.rc('font', size=FS)          # controls default text sizes
plt.rc('axes', titlesize=FS)     # fontsize of the axes title
plt.rc('axes', labelsize=FS)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=FS)    # fontsize of the tick labels
plt.rc('ytick', labelsize=FS)    # fontsize of the tick labels
plt.rc('legend', fontsize=FS)    # legend fontsize
plt.rc('figure', titlesize=FS)   # fontsize of the figure title

### Parameter definitions 
Here, the parameters that are found in the equations are given their value.

In [ ]:
#tensile stress
rS   = 0.3
Smin = 0.0001 
Smax = 0.3    
Scri = 0.15   
cS   = 0.0015
e    = 1.0
#water content
cW   = 0.001125
kW   = 0.05
dE   = 0.025
lP   = 0.039
kP   = 1.0
DW   = 0.3
#humidity of soil surface
rE   = 0.25
h    = 1.0
DE   = 0.15
#plant biomass
rP   = 0.25
dP   = 0.25
DP   = 0.015

### Root

In [ ]:
from scipy.optimize import fsolve
def func(par):
    S,W,E,P=par[0],par[1],par[2],par[3]
    return [rS*(S-Smin)*(S-Smax)*(Scri-S) + cS*(E-e),
            cW/(kW+S) - dE*(E-e)*W - lP*W*P/(kP+W),
            rE*(W - h)*(E-e),
            rP*P*(1.0-S)*W/(kP+W) - dP*P*P]
s1 = fsolve(func,[0,0,0,0.5])
s2 = fsolve(func,[0.3,1,1.2,0])
print(s1)
print(s2)

In [ ]:
from scipy.optimize import root
def func(par):
    S,W,E,P=par[0],par[1],par[2],par[3]
    return [rS*(S-Smin)*(S-Smax)*(Scri-S) + cS*(E-e),
            cW/(kW+S) - dE*(E-e)*W - lP*W*P/(kP+W),
            rE*(W - h)*(E-e),
            rP*P*(1.0-S)*W/(kP+W) - dP*P*P]
s = root(func,[0,3,1,1])
print(s)

### Simulation settings

In [ ]:
length    = 4096.0    # Length of the physical landscape
n         = 4096      # Size of the 2D grid
endtime   = 10000.0    # End time
dT        = 0.02      # Calculate time step
nplot     = 100
nsteps    = np.ceil(endtime/nplot/dT).astype(int)   #number of time steps
dX          = length/n   # Spatial step size
dY          = length/n   # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

### Defining the device that is used

In [ ]:
import pyopencl as cl
# Setting up the OpenCL context
DeviceNr = 1   # 0 = GTX 960M
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

### Initial values

In [ ]:
# The array is allocated on the GPU and the initial values are copied onto it
#Stress
S        = np.ones((n*n),dtype=float)*0.0
S_host   = S.astype(np.float32)
S_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=S_host)
#Water
W        = np.ones((n*n),dtype=float)*3.0
W_host   = W.astype(np.float32)
W_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=W_host)
#Evaporation
E        = (np.random.rand(n*n)>0.9996)*1.0+1.0
E_host   = E.astype(np.float32)
E_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=E_host)
#Plant
P        = np.ones((n*n),dtype=float)*1.0
P_host   = P.astype(np.float32)
P_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=P_host)

#Water content
#parameter ->array
Par_max=  1.2
Par_min=  0.8
Par_arr        = np.logspace(np.log10(Par_max),np.log10(Par_min),n).repeat(n)#np.logspace(np.log10(3.3),np.log10(0.3),n).repeat(n)#np.logspace(np.log10(3.3),np.log10(0.3),n).repeat(n)#0.85
Par_arr_host   = Par_arr.astype(np.float32)
Par_arr_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Par_arr_host)

# #Surface water diffusion coefficient
# #parameter2 ->array
# Par2_max=  0.2
# Par2_min=  0.1
# Par2_arr        = np.tile(np.linspace((Par2_min),(Par2_max),n),n)#np.linspace(Par2_max,Par2_min,n).repeat(n)#np.logspace(np.log10(3.3),np.log10(0.3),n).repeat(n)#np.logspace(np.log10(3.3),np.log10(0.3),n).repeat(n)#0.85
# Par2_arr_host   = Par2_arr.astype(np.float32)
# Par2_arr_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=Par2_arr_host)

In [ ]:
#parameter is X or Y ?
cl.enqueue_copy(queue, Par_arr_host, Par_arr_g)
plt.imshow(Par_arr_host.reshape(Grid_Width, Grid_Height),cmap='YlGn',extent=[0,length,0,length])
cbar=plt.colorbar(fraction=0.046,pad=0.04);

### Loading the functions d2_dxy2() and Periodicboundaries()

In [ ]:
with open('SpatialFunctions_iPy.cl', 'r') as myfile:
   SpatialFunctions = myfile.read()

### List of parameters for the OpenCL kernel.

In [ ]:
# Setting up the parameters for the Kernel
PassVars="rS,Smin,Smax,Scri,cS,e, cW,kW,dE,lP,kP,DW, rE,h,DE, rP,dP,DP, dX,dY,dT,Grid_Width,Grid_Height"

PassVals=eval(PassVars)
PassVars=PassVars.split(',')
Params=""

for ii in range(len(PassVals)):
    Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

### Defining the OpenCL simulation kernel

In [ ]:
ComputeCode = """

///////////////////////////////////////////////////////////////////////////////
// Simulation kernel
///////////////////////////////////////////////////////////////////////////////   

__kernel void SimulationKernel (__global float* S, __global float* W, __global float* E, __global float* P, __global float* Par_arr)
{

    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;

        if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
            {                
             float dSdt =   rS*(S[current]-Smin)*(S[current]-Smax)*(Scri-S[current]) + cS*(E[current]-e);
             float dWdt =   cW/(kW+S[current]) - dE*(E[current]-e)*W[current] - lP*W[current]*P[current]/(kP+W[current]) + DW*d2_dxy2(W);
             float dEdt =   rE*(W[current] - Par_arr[current])*(E[current]-e) + DE*d2_dxy2(E);
             float dPdt =   rP*P[current]*(1.0-S[current])*W[current]/(kP+W[current]) - dP*P[current]*P[current] + DP*d2_dxy2(P);
             
             S[current] = S[current] + dSdt*dT;
             W[current] = W[current] + dWdt*dT;
             E[current] = E[current] + dEdt*dT;
             P[current] = P[current] + dPdt*dT;
             
             //S[current]=(S[current]>0)*S[current];
             //W[current]=(W[current]>0)*W[current];
             //E[current]=(E[current]>0)*E[current];
             //P[current]=(P[current]>0)*P[current];
            }
            
        // HANDLE Boundaries
        else 
            {
             NeumannBoundaries(S);
             NeumannBoundaries(W);
             NeumannBoundaries(E);
             NeumannBoundaries(P);
            }

} // End SimulationKernel
"""

### Here the kernel is compiled

In [ ]:
program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

### The main simulation loop

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

# Setting up a progress bar for the simulation
print("Progress :");
PB = FloatProgress(min=0, max=nplot); display(PB) 

Ss=np.zeros((Grid_Width, Grid_Height, nplot))
Ws=np.zeros((Grid_Width, Grid_Height, nplot))
Es=np.zeros((Grid_Width, Grid_Height, nplot))
Ps=np.zeros((Grid_Width, Grid_Height, nplot))

# Set up simulation parameters
global_size = P_host.shape

# Start the timer:
start_time = time.time()

# Starting the loop
for ii in range(nplot):
    # The simulation
    # We store the state of the system for <NumPlot> different times.
    Ss[:,:,ii] = S_host.reshape(Grid_Width, Grid_Height)
    Ws[:,:,ii] = W_host.reshape(Grid_Width, Grid_Height)
    Es[:,:,ii] = E_host.reshape(Grid_Width, Grid_Height)
    Ps[:,:,ii] = P_host.reshape(Grid_Width, Grid_Height)
    
    for jj in range(nsteps):      
        program.SimulationKernel(queue, global_size, None, S_g, W_g, E_g, P_g, Par_arr_g)

    # Get the data from the GPU
    cl.enqueue_copy(queue, S_host, S_g)
    cl.enqueue_copy(queue, W_host, W_g)
    cl.enqueue_copy(queue, E_host, E_g)
    cl.enqueue_copy(queue, P_host, P_g)

#     print(ii)
    PB.value += 1 # signal to increment the progress bar

# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))

### Save data

In [ ]:
# from scipy import io
# import os
# if not os.path.exists("Data"):
#     os.makedirs("Data")
# io.savemat('Data/UV_data.mat', {'Plant': Ps[:,:,99],'Par_max': Par_max, 'Par_min': Par_min,
#                                 'Par2_max': Par2_max, 'Par2_min': Par2_min})
# # # load data: mathfn.mat from Matlab
# # # data = io.loadmat(matfn) 

In [ ]:
# from scipy import io
# # io.savemat('plant1.mat', {'plant1': APB})
# # io.savemat('plant11.mat', {'plant11': APB})
# #data = io.loadmat(matfn)
# np.save('Par13Ps.npy',Ps[:,:,:])
# np.save('Par13Ss.npy',Ss[:,:,:])
# np.save('Par13Ws.npy',Ws[:,:,:])
# np.save('Par13Es.npy',Es[:,:,:])

# Load data

In [ ]:
# Ps=np.load('Par13Ps.npy')
# P_host  = (Ps[:,:,nplot-1].reshape(n*n)).astype(np.float32)
# P_g     = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=P_host)

# # from scipy import io
# # io.savemat('Par13Ps.mat', {'Ps': Ps[:,:,nplot-1]})

In [ ]:
# Ss=np.load('Par13Ss.npy')
# Ws=np.load('Par13Ws.npy')
# Es=np.load('Par13Es.npy')
# S_host  = (Ss[:,:,nplot-1].reshape(n*n)).astype(np.float32)
# S_g     = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=S_host)
# W_host  = (Ws[:,:,nplot-1].reshape(n*n)).astype(np.float32)
# W_g     = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=W_host)
# E_host  = (Es[:,:,nplot-1].reshape(n*n)).astype(np.float32)
# E_g     = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=E_host)

### Plotting the results
Plant

In [ ]:
Plength    = 50
nticker = 5
# viridis, Reds, Greens
from matplotlib import ticker
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
im0 = ax.imshow(P_host.reshape(n,n),cmap='Reds',extent=[0,Plength,0,Plength],clim=(0.3,0.6));#, clim=(0,0.1));
#ax.set_title('S');
ax.set_xlabel('space, $x$ ($m$)');
ax.set_ylabel('space, $y$ ($m$)');
cbar=plt.colorbar(im0, ax=ax,fraction=0.046,pad=0.04);
# ax[0].set_yticks(np.linspace(0,length,nticker))
tick_locator = ticker.MaxNLocator(nbins=3)
cbar.locator = tick_locator
cbar.update_ticks()
# Xlabels = np.linspace(Par2_min,Par2_max,nticker).round(3)
# Ylabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,Plength,nticker))

# ax.set_xticklabels(Xlabels)
ax.set_yticks(np.linspace(0,Plength,nticker))
# ax.set_yticklabels(Ylabels)
fig.savefig('Fig6pB_WithCrack.pdf',bbox_inches='tight')

### Plotting the results in one dimension

#### Select one column

In [ ]:
npx = np.linspace(0,n,n)
fig1, ax = plt.subplots(1, 1, figsize=(10, 8))
line10, = ax.plot(npx, np.flipud(Ps[:,int(length/2),nplot-1]), lw=2, label='Plant'); 
line11, = ax.plot(npx, np.flipud(Ws[:,int(length/2),nplot-1]), lw=2, label='Water'); 
line12, = ax.plot(npx, np.flipud(Es[:,int(length/2),nplot-1]), lw=2, label='Evaporation'); 
line13, = ax.plot(npx, np.flipud(Ss[:,int(length/2),nplot-1]), lw=2, label='Stress'); 

ax.set_ylim(0, 1.5)
ax.set_ylabel('Plant Water Stress, $P W S$');

ax.set_xlim(0, length)
ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
ax.legend(fontsize=18)
ax.grid()


### SWEP: Mean Max Min

In [ ]:
Smax = np.ones(int(length),dtype=float)*0.0
Smin = np.ones(int(length),dtype=float)*0.0
Smean = np.ones(int(length),dtype=float)*0.0
Wmax = np.ones(int(length),dtype=float)*0.0
Wmin = np.ones(int(length),dtype=float)*0.0
Wmean = np.ones(int(length),dtype=float)*0.0
Emax = np.ones(int(length),dtype=float)*0.0
Emin = np.ones(int(length),dtype=float)*0.0
Emean = np.ones(int(length),dtype=float)*0.0
Pmax = np.ones(int(length),dtype=float)*0.0
Pmin = np.ones(int(length),dtype=float)*0.0
Pmean = np.ones(int(length),dtype=float)*0.0

for imax in range(int(length)):
    Smax[imax] = np.max(Ss[imax,:,nplot-1])
    Smin[imax] = np.min(Ss[imax,:,nplot-1])
    Smean[imax] = np.mean(Ss[imax,:,nplot-1])
    
    Wmax[imax] = np.max(Ws[imax,:,nplot-1])
    Wmin[imax] = np.min(Ws[imax,:,nplot-1])
    Wmean[imax] = np.mean(Ws[imax,:,nplot-1])

    Emax[imax] = np.max(Es[imax,:,nplot-1])
    Emin[imax] = np.min(Es[imax,:,nplot-1])
    Emean[imax] = np.mean(Es[imax,:,nplot-1])
    
    Pmax[imax] = np.max(Ps[imax,:,nplot-1])
    Pmin[imax] = np.min(Ps[imax,:,nplot-1])
    Pmean[imax] = np.mean(Ps[imax,:,nplot-1])

#### SWEP: Mean

In [ ]:
npx = np.linspace(0,n,n)
fig2, ax = plt.subplots(1, 1, figsize=(8, 4))

line10, = ax.plot(npx, np.flipud(Smean), ls='-', lw=3, label='Stress');
line11, = ax.plot(npx, np.flipud(Wmean), ls='-', lw=3, label='Water');
line12, = ax.plot(npx, np.flipud(Emean), ls='-', lw=3, label='Humidity');
line13, = ax.plot(npx, np.flipud(Pmean), ls='-', lw=3, label='Plant');

# ax.set_ylim(0.19, 0.61)
# ax.set_ylabel('Plant biomass, $P$ [$g/m^2$]');

ax.set_xlim(0, length)
ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
ax.legend(fontsize=16)
ax.grid()


#### Stress: Mean Max Min

In [ ]:
npx = np.linspace(0,n,n)
fig2, ax = plt.subplots(1, 1, figsize=(8, 4))

for isca in np.random.randint(0,int(length),3):
    ax.plot(npx, np.flipud(Ss[:,isca,nplot-1]),'b.', markersize=1.0);
# line10, = ax.plot(npx, np.flipud(Ss[:,int(length/2),nplot-1]),'b.', markersize=1.0); 
# line10, = ax.plot(npx, np.flipud(Ss[:,int(length/2),nplot-1]), lw=2, label='Stress'); 
# line11, = ax.plot(npx, np.flipud(Smax), ls='-', lw=1.5, color='green', label='Max'); 
# line12, = ax.plot(npx, np.flipud(Smin), ls='-', lw=1.5, color='green', label='Min');
line13, = ax.plot(npx, np.flipud(Smean), ls='-', lw=3, color='red', label='Mean');

# ax.set_ylim(-0.005, 0.305)
ax.set_ylabel('Stress, $S$ [$mm$]');

ax.set_xlim(0, length)
ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
# ax.legend(fontsize=18)
ax.grid()


#### Water: Mean Max Min

In [ ]:
npx = np.linspace(0,n,n)
fig2, ax = plt.subplots(1, 1, figsize=(8, 4))

for isca in np.random.randint(0,int(length),10):
    ax.plot(npx, np.flipud(Ws[:,isca,nplot-1]),'b.', markersize=1.0);
# line10, = ax.plot(npx, np.flipud(Ws[:,int(length/2),nplot-1]),'b.', markersize=1.0); 
# line10, = ax.plot(npx, np.flipud(Ws[:,int(length/2),nplot-1]), lw=2, label='Water'); 
line11, = ax.plot(npx, np.flipud(Wmax), ls='-', lw=1.5, color='green', label='Max'); 
line12, = ax.plot(npx, np.flipud(Wmin), ls='-', lw=1.5, color='green', label='Min');
line13, = ax.plot(npx, np.flipud(Wmean), ls='-', lw=3, color='red', label='Mean');

# ax.set_ylim(0.45, 1.55)
ax.set_ylabel('Water, $W$ [$mm$]');

ax.set_xlim(0, length)
ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
# ax.legend(fontsize=18)
ax.grid()


#### Humidity: Mean Max Min

In [ ]:
npx = np.linspace(0,n,n)
fig2, ax = plt.subplots(1, 1, figsize=(8, 4))

for isca in np.random.randint(0,int(length),10):
    ax.plot(npx, np.flipud(Es[:,isca,nplot-1]),'b.', markersize=1.0);
# line10, = ax.plot(npx, np.flipud(Es[:,int(length/2),nplot-1]),'b.', markersize=1.0); 
# line10, = ax.plot(npx, np.flipud(Es[:,int(length/2),nplot-1]), lw=2, label='Humidity'); 
line11, = ax.plot(npx, np.flipud(Emax), ls='-', lw=1.5, color='green', label='Max'); 
line12, = ax.plot(npx, np.flipud(Emin), ls='-', lw=1.5, color='green', label='Min');
line13, = ax.plot(npx, np.flipud(Emean), ls='-', lw=3, color='red', label='Mean');

# ax.set_ylim(0.99, 1.21)
ax.set_ylabel('Humidity, $E$ [$mm$]');

ax.set_xlim(0, length)
ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
# ax.legend(fontsize=18)
ax.grid()


#### Plant: Mean Max Min

In [ ]:
npx = np.linspace(0,n,n)
fig2, ax = plt.subplots(1, 1, figsize=(8, 4))

for isca in np.random.randint(0,int(length),10):
    ax.plot(npx, np.flipud(Ps[:,isca,nplot-1]),'.', color='gray', markersize=1.0);
# line10, = ax.plot(npx, np.flipud(Ps[:,int(length/2),nplot-1]),'b.', markersize=1.0); 
# line10, = ax.plot(npx, np.flipud(Ps[:,int(length/2),nplot-1]), lw=2, label='Plant'); 
line11, = ax.plot(npx, np.flipud(Pmax), ls='-', lw=1.5, color='#00CCFF', label='Max'); 
line12, = ax.plot(npx, np.flipud(Pmin), ls='-', lw=1.5, color='#00CCFF', label='Min'); 
line13, = ax.plot(npx, np.flipud(Pmean), ls='-', lw=3, color='red', label='Mean');

ax.set_ylim(0.18, 0.62)
# ax.set_ylabel('Plant biomass, $P$ [$g/m^2$]');

ax.set_xlim(0, length)
# ax.set_xlabel('Water content, $h$ [$g/m^3$]');
Xlabels = np.logspace(np.log10(Par_min),np.log10(Par_max),nticker).round(1)
ax.set_xticks(np.linspace(0,length,nticker))
ax.set_xticklabels(Xlabels)
# ax.legend(fontsize=18)
ax.grid()

fig2.savefig('Fig6pD_WithCrack.pdf',bbox_inches='tight')

In [ ]:
%reset

© 2021, ECNU, Shanghai, Kang Zhang.\
Follow Johan van de Koppel & Quan-Xing Liu.